In [2]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml
from scipy.optimize import fmin_l_bfgs_b


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
Data, labels = ml.loadFile('./Train.txt')
n, m = sum(labels == 1), sum(labels == 0)
#print('Prima ', n/m)
numpy.random.seed(0)
indexes = numpy.random.permutation(1200)
Data_s = Data[:, indexes]
label_s = labels[indexes]
ns, ms = sum(label_s == 1), sum(label_s == 0)
#print('Dopo :', ns/ms)

In [4]:
(DTR, LTR), (DTE, LTE) = ml.split_db_2to1(Data_s, label_s, seed=42)
n1, n0 = sum(LTR == 1), sum(LTR == 0)

In [8]:
def L_dual_wrapper(H_hat):
    def L_dual(alpha):
        one_vect = numpy.ones((len(alpha)), dtype='int32')
        L_d = 1/2 * numpy.dot( alpha.T, numpy.dot( H_hat, alpha ) ) - numpy.dot( alpha.T, one_vect )
        grad_L_d = numpy.dot( H_hat, alpha ) - one_vect

        v = numpy.array((L_d, grad_L_d), dtype=object)

        return v
    return L_dual

def polynomialSVM(DTR, LTR, DTE, params):
    K, C, d, c = params[0], params[1], params[2], params[3]
    
    x0 = numpy.zeros((DTR.shape[1]), dtype='int32')
    H_hat = ml.compute_H_hat2(DTR, LTR, K**2, c, d)

    boundaries = []
    for i in range(DTR.shape[1]):
        boundaries.append((0, C))

    alpha, f, dictionary = fmin_l_bfgs_b(L_dual_wrapper(H_hat), x0, bounds=boundaries, factr=1.0)

    S = ml.compute_score(alpha, DTR, LTR, DTE, K**2, c, d)

    return list(S)

In [18]:
def kfold(D, L, fold, params, app):
    error = 0
    N = int(D.shape[1]/fold) #numero di elementi per ogni fold
    numpy.random.seed(0) #imposto il seed del generatore di numeri casuali -> in tal modo genererò la stessa sequenza di numeri casuali aventi seed uguale
    indexes = numpy.random.permutation(D.shape[1]) #genero una sequenza di numeri casuali che vanno da 0 al num_di_campioni
    
    LTE_final = []
    llr_final = []
    for j in range(fold):
        test_indexes = indexes[(j*N):((j+1)*N)] #selezioni gli indici che identificano i campioni (casuali) del test set
        if(j > 0): #se il test set non è preso dalla prima fold (--> il test set è una fold intermedia o l'ultima fold)
            left_indexes = indexes[0:(j*N)] #allora prendo tutti gli indici che stanno a sinistra di tale fold
        else: #se il test set è preso dalla prima fold
            right_indexes = indexes[((j+1)*N):] #prendo tutti gli indici a destra della prima fold

        if(j == 0): #se il test set è preso dalla prima fold
            train_indexes = right_indexes #assegno agli indici di training quelli che stanno a destra della prima fold
        elif(j == fold-1): #se il test set è preso dall'ultima fold
            train_indexes = left_indexes #assegno agli indici di training quelli che stanno a sinistra dell'ultima fold
        else: #in questo caso il test set è preso da una fold intermedia
            train_indexes = numpy.hstack((left_indexes, right_indexes)) #pertanto assegno agli indici di training quelli appartenenti alle fold di sinistra e di destra

        DTR = D[:, train_indexes]  #definisco insieme di training e di testing
        LTR = L[train_indexes]
        DTE = D[:, test_indexes]
        LTE = L[test_indexes]
        LTE_final.extend(LTE)
        llr_final.extend(polynomialSVM(DTR, LTR, DTE, params))
        
    CM = ml.compute_optimal_B_decision(app, llr_final, LTE_final)
    sensitivity = (1-ml.compute_FNR(CM)) # TPR
    specificity = (1-ml.compute_FPR(CM)) # TNR
        
    app_bayes_risk=ml.compute_Bayes_risk(CM, app)
    DCF = ml.compute_norm_Bayes(app_bayes_risk, app)

    minDCF= ml.compute_min_DCF(llr_final, app, LTE_final)
    error = 1-(CM[0, 0]+CM[1,1])/(len(LTE_final))

    print("\-/ \-/ \-/ \-/ \-/ ")
    print("Polynomial SVM error:", error)
    print(f'{app} DCF: {round(DCF, 3)}, minDCF: {round(minDCF, 3)}')
    print(f'Sensitivity (TPR): {round(sensitivity, 3)}, Specificity (TNR): {round(specificity, 3)}')
    print('CM\n', CM)
    print("/-\ /-\ /-\ /-\ /-\ ")

In [9]:
kfold(Data_s, label_s, 5, [0, 1, 2, 0], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.31166666666666665
[0.5, 1, 1] DCF: 0.5171076863429291 minDCF: 0.4325869923427353
Sensitivity (TPR): 0.8070175438596492  Specificity (TNR):  0.6758747697974217
CM
 [[734.  22.]
 [352.  92.]]
/-\ /-\ /-\ /-\ /-\ 


In [10]:
kfold(Data_s, label_s, 5, [0, 0.1, 2, 0], [0.5, 1, 1])
kfold(Data_s, label_s, 5, [0, 0.01, 2, 0], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.1433333333333333
[0.5, 1, 1] DCF: 0.40176407870504993 minDCF: 0.38014926819811956
Sensitivity (TPR): 0.7280701754385965  Specificity (TNR):  0.8701657458563536
CM
 [[945.  31.]
 [141.  83.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.08333333333333337
[0.5, 1, 1] DCF: 0.34331685567509934 minDCF: 0.3239798391005137
Sensitivity (TPR): 0.7192982456140351  Specificity (TNR):  0.9373848987108655
CM
 [[1018.   32.]
 [  68.   82.]]
/-\ /-\ /-\ /-\ /-\ 


In [13]:
kfold(Data_s, label_s, 5, [0, 0.01, 2, 1], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.32166666666666666
[0.5, 1, 1] DCF: 0.5203062905883493 minDCF: 0.40913056120965396
Sensitivity (TPR): 0.8157894736842105  Specificity (TNR):  0.6639042357274402
CM
 [[721.  21.]
 [365.  93.]]
/-\ /-\ /-\ /-\ /-\ 


In [11]:
kfold(Data_s, label_s, 5, [1, 0.1, 2, 0], [0.5, 1, 1])
kfold(Data_s, label_s, 5, [1, 0.01, 2, 0], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.2908333333333334
[0.5, 1, 1] DCF: 0.6432586992342735 minDCF: 0.5138121546961326
Sensitivity (TPR): 0.6403508771929824  Specificity (TNR):  0.7163904235727441
CM
 [[778.  41.]
 [308.  73.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.15916666666666668
[0.5, 1, 1] DCF: 0.2936415624697102 minDCF: 0.2853542696520306
Sensitivity (TPR): 0.868421052631579  Specificity (TNR):  0.8379373848987108
CM
 [[910.  15.]
 [176.  99.]]
/-\ /-\ /-\ /-\ /-\ 


In [12]:
kfold(Data_s, label_s, 5, [10, 0.1, 2, 0], [0.5, 1, 1])
kfold(Data_s, label_s, 5, [10, 0.01, 2, 0], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.31499999999999995
[0.5, 1, 1] DCF: 0.4815353300378017 minDCF: 0.4185809828438499
Sensitivity (TPR): 0.8508771929824561  Specificity (TNR):  0.6675874769797422
CM
 [[725.  17.]
 [361.  97.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.2858333333333334
[0.5, 1, 1] DCF: 0.449306969080159 minDCF: 0.35426965203062905
Sensitivity (TPR): 0.8508771929824561  Specificity (TNR):  0.6998158379373849
CM
 [[760.  17.]
 [326.  97.]]
/-\ /-\ /-\ /-\ /-\ 


In [15]:
kfold(Data, labels, 5, [0, 0.01, 2, 0], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.48504201680672265
[0.5, 1, 1] DCF: 1.0847800157032872 minDCF: 0.8517972207826874
Sensitivity (TPR): 0.3873325213154689  Specificity (TNR):  0.5278874629812438
CM
 [[4278.  503.]
 [3826.  318.]]
/-\ /-\ /-\ /-\ /-\ 


In [21]:
data_g = ml.gaussianize(Data)
data_z = ml.z_normalization(Data)

In [22]:
kfold(data_g, labels, 5, [0, 1, 2, 0], [0.5, 1, 1])
kfold(data_g, labels, 5, [1, 1, 2, 0], [0.5, 1, 1])
kfold(data_g, labels, 5, [0, 1, 2, 1], [0.5, 1, 1])
kfold(data_g, labels, 5, [1, 1, 2, 1], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.03731092436974792
[0.5, 1, 1] DCF: 0.221, minDCF: 0.216
Sensitivity (TPR): 0.8, Specificity (TNR): 0.979
CM
 [[7935.  164.]
 [ 169.  657.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.025210084033613467
[0.5, 1, 1] DCF: 0.202, minDCF: 0.154
Sensitivity (TPR): 0.806, Specificity (TNR): 0.992
CM
 [[8038.  159.]
 [  66.  662.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.01971988795518209
[0.5, 1, 1] DCF: 0.159, minDCF: 0.132
Sensitivity (TPR): 0.848, Specificity (TNR): 0.994
CM
 [[8053.  125.]
 [  51.  696.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.01971988795518209
[0.5, 1, 1] DCF: 0.159, minDCF: 0.131
Sensitivity (TPR): 0.848, Specificity (TNR): 0.994
CM
 [[8053.  125.]
 [  51.  696.]]
/-\ /-\ /-\ /-\ /-\ 


In [23]:
kfold(data_z, labels, 5, [0, 1, 2, 0], [0.5, 1, 1])
kfold(data_z, labels, 5, [1, 1, 2, 0], [0.5, 1, 1])
kfold(data_z, labels, 5, [0, 1, 2, 1], [0.5, 1, 1])
kfold(data_z, labels, 5, [1, 1, 2, 1], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.0328291316526611
[0.5, 1, 1] DCF: 0.246, minDCF: 0.227
Sensitivity (TPR): 0.766, Specificity (TNR): 0.988
CM
 [[8003.  192.]
 [ 101.  629.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.024649859943977614
[0.5, 1, 1] DCF: 0.229, minDCF: 0.135
Sensitivity (TPR): 0.776, Specificity (TNR): 0.996
CM
 [[8068.  184.]
 [  36.  637.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.020056022408963536
[0.5, 1, 1] DCF: 0.17300000000000001, minDCF: 0.111
Sensitivity (TPR): 0.8320000000000001, Specificity (TNR): 0.995
CM
 [[8063.  138.]
 [  41.  683.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Polynomial SVM error: 0.020056022408963536
[0.5, 1, 1] DCF: 0.17300000000000001, minDCF: 0.113
Sensitivity (TPR): 0.8320000000000001, Specificity (TNR): 0.995
CM
 [[8063.  138.]
 [  41.  683.]]
/-\ /-\ /-\ /-\ /-\ 
